In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# general
import numpy as np
import pandas as pd
import math
# import matplotlib.pyplot as plt
import os
import shutil
import pickle

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

# import custom libraries
import sys
# sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
# import hk_utils

# folder paths
ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
ADD_DATA_proc = "C:/DATA/data/processed/"


PATH_PAPER = "C:\\DATA\\Tasks\\220704\\Alternate-Transactions-Articles-LaTeX-template\\images\\"


PATH_SYS="/mlodata1/hokarami/tedam/"

In [2]:
# sudo conda install -c conda-forge datasets transformers evaluate --name paper2022

In [4]:
from datasets import load_dataset
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Downloading: 100%|██████████| 3.50k/3.50k [00:00<00:00, 2.23MB/s]
Downloading: 100%|██████████| 576M/576M [01:28<00:00, 6.55MB/s] 
Downloading: 100%|██████████| 21.1M/21.1M [00:02<00:00, 7.12MB/s]
Downloading: 100%|██████████| 53.0M/53.0M [00:07<00:00, 6.80MB/s]
Downloading: 100%|██████████| 286M/286M [00:39<00:00, 7.21MB/s] 
Downloading: 100%|██████████| 9.65M/9.65M [00:01<00:00, 7.16MB/s]
Downloading: 100%|██████████| 17.7M/17.7M [00:02<00:00, 7.77MB/s]
Downloading: 100%|██████████| 330M/330M [00:46<00:00, 7.17MB/s] 
Downloading: 100%|██████████| 18.7M/18.7M [00:02<00:00, 7.77MB/s]
Downloading: 100%|██████████| 36.2M/36.2M [00:05<00:00, 6.79MB/s]

Dataset eli5 downloaded and prepared to /home/hokarami/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


In [6]:
eli5 = eli5.train_test_split(test_size=0.2)

In [7]:
eli5["train"][0]

{'q_id': '4zvqmr',
 'title': 'How can non-locality avoid violating causality?',
 'selftext': 'I was watching a PBS Space Time video talking about the Quantum Eraser experiment. In the [video](_URL_0_) the host mentions around the 9:30 mark that non-locality doesn\'t violate causality and that this is tied to the phenomena of quantum entanglement where instantaneous "communication" takes place between two entangled particles. How can this avoid violating causality when you have information traveling instantaneously? ',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['d6zby2c'],
  'text': ['The trick is information is never actually transmitted via entanglement. Thus, causality is never broken since you can\'t actually influence anyone outside your lightcone. In other words, entanglement can only ever produce correlated results, but each result is truly independent and random and you\'d never know something profoundly weird occurred until you met with the person with t

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [9]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '4zvqmr',
 'title': 'How can non-locality avoid violating causality?',
 'selftext': 'I was watching a PBS Space Time video talking about the Quantum Eraser experiment. In the [video](_URL_0_) the host mentions around the 9:30 mark that non-locality doesn\'t violate causality and that this is tied to the phenomena of quantum entanglement where instantaneous "communication" takes place between two entangled particles. How can this avoid violating causality when you have information traveling instantaneously? ',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['d6zby2c'],
 'answers.text': ['The trick is information is never actually transmitted via entanglement. Thus, causality is never broken since you can\'t actually influence anyone outside your lightcone. In other words, entanglement can only ever produce correlated results, but each result is truly independent and random and you\'d never know something profoundly weird occurred until you met with the person wit

In [10]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [11]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (741 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors
Map (num_proc=4):  25%|██▌       | 1000/4000 [00:01<00:03, 951.64 examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1470 > 512). Running this sequence through the model will result in indexing errors
Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]             Token indices sequence length is longer than the sp

In [12]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [14]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [18]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_mlm_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

/opt/conda/envs/paper2022/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/hokarami/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "stevhliu/my_awesome_eli5_mlm_model")
mask_filler(text, top_k=3)

# Load